In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf

In [ ]:
tickers = ['BBCA.JK', 'TLKM.JK', 'ANTM.JK', 'BMRI.JK', 'BRPT.JK', '^JKSE']

data = yf.download(
    tickers,
    start='2020-08-01',
    end='2025-08-16',
    group_by='ticker'
)

# Membuat dictionary untuk menyimpan data masing-masing ticker
stocks_data = {}

for ticker in tickers:
    try:
        df = data[ticker][['Open', 'High', 'Low', 'Close', 'Volume']].copy()

        # 1. Hitung Daily Returns (% perubahan harga harian)
        df['Daily_Return'] = df['Close'].pct_change()

        # 2. Hitung Moving Averages (MA 5, MA 15, MA 20)
        df['MA_5'] = df['Close'].rolling(window=5).mean()
        df['MA_15'] = df['Close'].rolling(window=15).mean()
        df['MA_20'] = df['Close'].rolling(window=20).mean()

        # 3. Hitung Volatility (Standar Deviasi Return 20 hari)
        df['Volatility_20D'] = df['Daily_Return'].rolling(window=20).std()
        df['Ticker'] = ticker
        df = df.reset_index()
        stocks_data[ticker] = df
        print(f"Successfully processed data for {ticker}")

    except KeyError:
        print(f"Failed to retrieve data for {ticker}")

combined_df = pd.concat(stocks_data, ignore_index=True)
print(combined_df[combined_df['Ticker'] == 'TLKM.JK'].tail())

/tmp/ipython-input-1470403879.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  6 of 6 completed

Successfully processed data for BBCA.JK
Successfully processed data for TLKM.JK
Successfully processed data for ANTM.JK
Successfully processed data for BMRI.JK
Successfully processed data for BRPT.JK
Successfully processed data for ^JKSE
Price       Date    Open    High     Low   Close     Volume  Daily_Return  \
2417  2025-08-11  2990.0  3010.0  2970.0  2990.0   74978300      0.017007   
2418  2025-08-12  3050.0  3200.0  3010.0  3180.0  217607800      0.063545   
2419  2025-08-13  3220.0  3420.0  3210.0  3360.0  465722400      0.056604   
2420  2025-08-14  3370.0  3490.0  3370.0  3420.0  232557200      0.017857   
2421  2025-08-15  3360.0  3410.0  3310.0  3310.0  182893900     -0.032164   

Price    MA_5        MA_15   MA_20  Volatility_20D   Ticker  
2417   2980.0  2918.000000  2870.0        0.015749  TLKM.JK  
2418   3010.0  2940.000000  2897.0        0.019437  TLKM.JK  
2419   3084.0  2974.000000  2932.5        0.022032  TLKM.JK  
2420   3178.0  3012.666667  2965.0        0.020673 

In [ ]:
combined_df.to_csv('/content/combined_data.csv', index=False)

In [ ]:
import tweepy
import time

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAC1a3gEAAAAA%2BUzvBFVbl7B02V8f%2FVG9TlOi%2F2M%3DcbeQFFHStmTa1ZbH4ehI8Yoq8MkaI1LtFxAf48wzbrJDMdxzHO"

client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

query = "Saham BMRI lang:id -is:retweet"

try:
  response = client.search_recent_tweets(
      query=query,
      max_results= 32,
      tweet_fields=["created_at", "public_metrics", "author_id"]
  )
  print(f"Berhasil mengambil {len(response.data)} tweet.")
except tweepy.errors.TooManyRequests:
    print("Rate limit tercapai. Menunggu selama 15 menit...")
    time.sleep(15 * 60)  # Tunggu selama 15 menit (15 * 60 detik)
    print("Waktu tunggu selesai, coba jalankan skrip lagi.")



Berhasil mengambil 23 tweet.


In [ ]:
if response.data:
    tweets_data = []
    for tweet in response.data:
        tweet_info = {
            "created_at": tweet.created_at,
            "author_id": tweet.author_id,
            "text": tweet.text,
            "retweet_count": tweet.public_metrics['retweet_count'],
            "like_count": tweet.public_metrics['like_count'],
            "reply_count": tweet.public_metrics['reply_count']
        }
        tweets_data.append(tweet_info)

    df_BMRI = pd.DataFrame(tweets_data)
    print(f"Berhasil mengambil {len(df_BMRI)} tweet.")
    print(df_BMRI)

Berhasil mengambil 23 tweet.
                  created_at            author_id  \
0  2025-08-14 12:19:32+00:00  1006840774502146049   
1  2025-08-14 10:39:38+00:00  1720665183188922368   
2  2025-08-14 03:19:59+00:00  1260798424393211904   
3  2025-08-14 00:41:02+00:00            238091066   
4  2025-08-13 09:21:58+00:00             36875940   
5  2025-08-13 09:09:04+00:00  1720665183188922368   
6  2025-08-13 06:35:03+00:00            238091066   
7  2025-08-13 02:37:27+00:00  1788855037684596736   
8  2025-08-13 01:58:18+00:00            454783366   
9  2025-08-13 00:33:19+00:00   886784783434436608   
10 2025-08-12 12:52:14+00:00  1059424005919191041   
11 2025-08-12 11:35:16+00:00  1501237081640554504   
12 2025-08-12 10:05:19+00:00   847372542830444544   
13 2025-08-12 09:53:32+00:00           3870150980   
14 2025-08-12 06:49:32+00:00  1450395130792329223   
15 2025-08-12 05:49:09+00:00            238091066   
16 2025-08-12 04:51:13+00:00            228201041   
17 2025-08-12 04:

In [ ]:
df_BMRI.to_csv('/content/Sentiment_BMRI.csv', index=False)